In [24]:
import numpy as np
import mat73
from datetime import datetime, timedelta
import logging as log

import wfdb
from wfdb import processing
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np

from tqdm import tqdm
import pyhrv.time_domain as td

In [25]:
class AdInstrumentData:
    
    def __init__(self, mat_files):
        # matfiles: can load several mat files
        
        data_dict = mat73.loadmat(mat_files[0])
        # extract start time
        data_starts = data_dict['record_meta']['data_start']
        self.__start_time = self.__days2datetime(data_starts[0]) if isinstance(data_starts, list) else self.__days2datetime(data_starts)
        
        # extract data frequency
        tick_dts = data_dict['record_meta']['tick_dt']
        self.__fs = 1/tick_dts[0] if isinstance(tick_dts, list) else 1/tick_dts

        from operator import itemgetter
        self.__template = 'data__chan_'
        all_ticks = {}
        
        def extract_ticks(data, all_ticks):
            # combine all ticks
            
            tmp_keys = []
            for key in data.keys():
                if self.__template in key:
                    tmp_keys += [[int(k) for k in key.split('_') if k.isnumeric()] + [key]]
            tmp_keys = sorted(tmp_keys, key=itemgetter(0, 1))

            for l_key in tmp_keys:
                key = self.__template + str(l_key[0])
                if type(data[l_key[-1]]) == np.ndarray:
                    all_ticks[key] = np.concatenate((all_ticks[key], data[l_key[-1]])) if key in all_ticks else data[l_key[-1]]

        extract_ticks(data_dict, all_ticks)
        for file in mat_files[1:]:
            extract_ticks(mat73.loadmat(file), all_ticks)
        self.__all_ticks = all_ticks
        self.__tick_len = len(list(all_ticks.values())[0])
        
        self.__stop_time = self.__start_time + timedelta(seconds=len(self.all_ticks[self.__template+'1'])/self.__fs)
        
        self.__date_format = "%Y-%m-%d %H:%M:%S.%f"
        
        log.info(f'Labchart data from {str(self.__start_time)} to {str(self.__stop_time)}')
        
    def __days2datetime(self, days):
        # days, ex: array(739071.96513903)
        reference_date = datetime(1, 1, 1)
        return reference_date + timedelta(days=float(days)-367)
    
    def __tick_pos2datetime(self, tick_pos:int):
        """
        args:
            tick_pos: position of tick
        return:
            date_time in datetime format
        """
        seconds = tick_pos / self.__fs
        return self.__start_time + timedelta(seconds=seconds)
    
    def __datetime2tick_pos(self, date_time):
        """
        args:
            date_time: date and time in sting or datetime data type
        return:
            position of ticks
        """
        if isinstance(date_time, str):
            date_time = datetime.strptime(date_time, self.__date_format)
        if isinstance(date_time, datetime):
            return round((date_time - self.__start_time).total_seconds() * self.__fs)
        return None
    
    @property
    def start_time(self):
        return self.__start_time
    
    @property
    def finish_time(self):
        return self.__stop_time
    
    @property
    def fs(self):
        return self.__fs
    
    @property
    def all_ticks(self):
        return self.__all_ticks
    
    @property
    def tick_len(self):
        return self.__tick_len
    
    @property
    def channels(self):
        return [c[len(self.__template):] for c in self.__all_ticks.keys()]
        
    def __get_ecg_range(self, channel, from_time=None, to_time=None, duration=3600):
        """
        from_time : time to start
        to_time: time to finish
        duration : duration in seconds, default = 1 hours = 60 * 60 seconds
        channel : start from 1
        return: validity(%), ticks, from_time, to_time
        """
        if from_time is None:
            from_time = self.__start_time
        start_tick = self.__datetime2tick_pos(from_time)
        if to_time is None and duration:
            finish_tick = start_tick + duration
            to_time = self.__tick_pos2datetime(finish_tick)
        elif to_time is not None:
            finish_tick = self.__datetime2tick_pos(to_time)
        if finish_tick > self.__tick_len:
            finish_tick = self.__tick_len
            to_time = self.__tick_pos2datetime(finish_tick)
        
        
        if start_tick >= self.__tick_len or start_tick >= finish_tick:
            return None
        
        tick_channel = self.__template + str(channel)
        validity_channel = self.__template + str(channel + 1 if (channel + 1) % 3 == 0 else channel + 2)
        
        ticks = self.__all_ticks[tick_channel][start_tick:finish_tick]
        validity = self.__all_ticks[validity_channel][start_tick:finish_tick]            
        
        
        # calculate validity level
        val_level = np.mean(validity)
        # remove ticks with invalid recording
        condition = validity < 0.5
        filtered_ticks = ticks[~condition]

        return val_level, filtered_ticks, from_time, to_time
        
    def get_data(self, channel, from_time=None, to_time=None, duration=3600):
        """
        from_time : time to start
        to_time: time to finish
        duration : duration in seconds, default = 1 hours = 60 * 60 seconds
        channel : start from 1
        return: validity(%), ticks, from_time, to_time
        """
        val_level, signal, from_time, to_time = self.__get_ecg_range(channel, from_time, to_time, duration)
        
        # GET PEAKS
        # rqrs config file for mouse
        # using PhysioZoo setup
        hr = 608
        qs = 0.00718
        qt = 0.03
        QRSa = 1090
        QRSamin = 370
        rr_min = 0.05
        rr_max = 0.24
        window_size_sec = 0.005744 # 0.8*QS

        # adjusting peaks location
        peaks_window = 17
        th = 0.5
        # Use the maximum possible bpm as the search radius
        # Ostergaard G, Hansen HN, Ottesen JL. Physiological, Hematological, and Clinical Chemistry Parameters, Including Conversion Factors. In: Hau J, Schapiro SJ, editors. Handbook of laboratory animal science, Volume I: Essential Principles and Practices. 3rd ed. Vol. 1. Boca Raton, FL: CRC Press; 2010. pp. 667–707.
        min_bpm = 310
        max_bpm = 840
        search_radius = int(self.__fs * 60 / max_bpm)

        # Use the GQRS algorithm to detect QRS locations in the first channel
        try:
            qrs_inds = processing.qrs.gqrs_detect(sig=signal, 
                                                  fs=self.__fs, 
                                                  RRmin=rr_min, 
                                                  RRmax=rr_max,
                                                  hr=hr, 
            #                                       QS=qs, 
                                                  QT=qt, 
                                                  QRSa=QRSa, 
            #                                       QRSamin=QRSamin
                                                 )
        except Exception as e:
            log.warning(f'Cannot identify peaks,\n {e}')
            qrs_inds = []
        
        # Correct the peaks shifting them to local maxima
        try:
            peaks = processing.peaks.correct_peaks(
                signal,
                peak_inds=qrs_inds,
                search_radius=search_radius, 
                smooth_window_size=peaks_window
            )
        except Exception as e:
            log.warning(f'Cannot correct peaks,\n {e}')
            peaks = qrs_inds
        
    
        # GET NNI, THEN FILTER NNI and PEAKS
        nni = np.diff(peaks)
        condition = (nni >= rr_min*self.__fs) & (nni <= rr_max*self.__fs)
        nni = nni[condition]
        
        # filter nni within mean +- std2x
        nni_mean = np.mean(nni)
        std2x = 2 * np.std(nni)
        condition = (nni >= nni_mean-std2x) & (nni <= nni_mean+std2x)
        nni = nni[condition]
        
        # get activity in lower data channel
        tick_channel = self.__template + str(channel-1)
        start_tick = self.__datetime2tick_pos(from_time)
        end_tick = self.__datetime2tick_pos(to_time)
        activity = self.__all_ticks[tick_channel][start_tick:end_tick]

        return {"val_level": val_level, "nni": nni, "from_time": from_time, "to_time":to_time, "activity": activity}
    
    

In [26]:
# Load all files
import os

def get_file_list(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".mat")]


## Analysis mouse 1

In [21]:
# setup
log.basicConfig(level=log.INFO)

# duration = 3600 #1 hour
# sliding_window = 1800 # 30 minutes
duration = 600 # 10 minutes
sliding_window = 300 # 5 minutes

minimum_data_validity = 0.3
minimum_nni_count = 10

# calculated = {'hr_means':[], 'activity':[], 'val_level':[], 'from_time':[], 'to_time':[], 'channel':[]}


In [22]:
import json
channels = [2]
directory = '/Volumes/Aswaty Nur/Pilot Project 1 (2022)/Converted_telemetry_20221026_mat'
log_file = 'log_project1.txt'

loaded_files = []
try:
    with open(log_file, 'r') as f:
        loaded_files = json.load(f)
except:
    pass
    
def update_log(file, loaded_files):
    loaded_files += [file]
    with open(log_file, 'w') as f:
        json.dump(loaded_files, f)

In [27]:
import pandas as pd

for file in get_file_list(directory):
    if file in loaded_files:
        continue

    raw_data = AdInstrumentData([file])

    start_time = raw_data.start_time
    finish_time = start_time + timedelta(seconds=duration)

    timer = 0

    while start_time < raw_data.finish_time:
        for channel in channels:
            if finish_time > raw_data.finish_time:
                finish_time = raw_data.finish_time
            ecg = raw_data.get_data(channel, from_time=start_time, to_time=finish_time)
#             hr_parameter = td.hr_parameters(nni=ecg['nni'])
#             calculated['hr_means'] += [hr_parameter[0]]
            calculated['hr_means'] += [ecg['nni']]
            calculated['activity'] += [np.mean(ecg['activity'])]
            calculated['val_level'] += [ecg['val_level']]
            calculated['from_time'] += [ecg['from_time']]
            calculated['to_time'] += [ecg['to_time']]
            calculated['channel'] += [channel]
        start_time = start_time + timedelta(seconds=sliding_window)
        finish_time = start_time + timedelta(seconds=duration)

        if timer%6==0: print('.', end='')
        timer += 1

    update_log(file, loaded_files)
    df_calculated = pd.DataFrame(calculated)
    df_calculated.to_csv('actogram_1_10min_5min.csv')
    

INFO:root:Labchart data from 2022-11-09 22:33:11.401225 to 2022-11-10 10:33:11.401225


........................

INFO:root:Labchart data from 2022-12-09 10:37:18.597118 to 2022-12-09 22:37:18.597118


........................

INFO:root:Labchart data from 2022-12-09 22:37:22.139626 to 2022-12-10 10:37:22.139626


..........

 zero-size array to reduction operation fmin which has no identity
 v cannot be empty


..............

In [1]:
import pandas as pd
df_calculated = pd.DataFrame(calculated)

df_calculated.to_csv('actogram_1_10min_5min.csv')

NameError: name 'calculated' is not defined

In [4]:
import pandas as pd

df_calculated = pd.read_csv('actogram_1_10min_5min.csv')



{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100:

In [9]:
tmp = df_calculated.to_dict()


In [16]:
calculated = {}
rows = ['hr_means', 'activity', 'val_level', 'from_time', 'to_time', 'channel']
for row_name, row_data in df_calculated.to_dict().items():
    if row_name in rows:
        calculated[row_name] = list(row_data.values())

calculated

{'hr_means': ['[231 227 231 ... 195 203 203]',
  '[225 231 228 ... 186 185 184]',
  '[200 205 211 ... 200 207 197]',
  '[185 192 191 ... 230 234 233]',
  '[189 189 192 ... 239 222 240]',
  '[239 232 228 228 240 228 235 235 236 232 240 232 240 228 238 220 240 227\n 237 230 235 238 236 224 211 230 233 240 226 219 217 217 227 238 226 227\n 223 219 204 214 217 221 223 217 217 217 219 220 214 210 227 229 224 236\n 238 225 215 227 237 232 235 229 236 238 236 237 235 231 235 225 227 235\n 240 236 239 227 236 226 228 234 238 232 237 235 232 239 233 239 237 240\n 237 235 239 228 235 230 230 240 236 229 239 233 240 231 240 237 236 233\n 237 234 228 228 229 234 236 237 229 225 238 234 235 232 235 236 238 239\n 227 229 236 238 240 240 230 240 236 224 235 232 234 238 237 235 231 236\n 230 231 240 236 240 240 238 234 237 239 236 240 238 238 238 236 229 238\n 236 236 231 230 231 225 216 235 239 234 237 231 229 224 223 215 237 235\n 240 239 231 236 235 232 238 240 239 240 229 237 239 209 231 229 230 2

In [30]:
list(calculated['hr_means'][0])

['[',
 '2',
 '3',
 '1',
 ' ',
 '2',
 '2',
 '7',
 ' ',
 '2',
 '3',
 '1',
 ' ',
 '.',
 '.',
 '.',
 ' ',
 '1',
 '9',
 '5',
 ' ',
 '2',
 '0',
 '3',
 ' ',
 '2',
 '0',
 '3',
 ']']